In [18]:
import seaborn as sns
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from torchvision import models
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import torch.nn.functional as F
#from einops import rearrange, repeat
#from einops.layers.torch import Rearrange
import math
import numpy as np
import time
from torch import einsum
import cv2
import scipy.misc
import utils
import tqdm
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
import pandas as pd

file_path = "squat_data.csv"

df = pd.read_csv(file_path)

print(df.head())

   updown       UWB     vel_x     vel_y     vel_z     accel_x     accel_y  \
0       0  1.711766  0.000000  0.000000  0.000000   11.828938  -47.956334   
1       0  1.711567  0.032885 -0.103377  0.085973  161.323707  -58.555513   
2       0  1.707675  0.814416 -0.361158 -0.095207 -178.527859  131.141650   
3       0  1.706877 -0.014801  0.076857 -0.414512  167.629076 -131.972570   
4       0  1.702809  0.778336 -0.387729 -0.164145 -152.167755   50.021166   

      accel_z    gyro_x    gyro_y    gyro_z  
0   62.560536  0.000000  0.000000  0.000000  
1  -18.781445  1.277550  0.196830 -0.315507  
2 -121.957382  0.458584  2.095545 -2.194921  
3  115.180361 -2.522667  1.956186  0.759899  
4   -3.156194  1.050402  2.661206 -2.515833  


In [23]:
df = df.drop(columns=['vel_x', 'vel_y', 'vel_z'])

In [24]:
tmp = df['updown'].shift(1).fillna(0)
squat_ends = df[(tmp == 1) & (df['updown'] == 0)].index
print(f"Total squat ends: {len(squat_ends)}")

squat_sets = []

start_index = 0
for end_index in squat_ends:
    one_set = df.iloc[start_index:end_index+1]

    if len(one_set) > 140:
        one_set = one_set.tail(140)
        squat_sets.append(one_set)
    else:
        print(f"Set too short: {len(one_set)}")

    #squat_sets.append(one_set)
    start_index = end_index + 1

print(f"Total sets: {len(squat_sets)}")

Total squat ends: 3000
Set too short: 140
Set too short: 140
Set too short: 138
Set too short: 135
Set too short: 139
Set too short: 140
Set too short: 138
Total sets: 2993


In [ ]:
# 한 sample당 15frames (현재 포함 과거 10개, 미래 5개), stride 5
X = []
y = []
for one_set in squat_sets:
    for i in range(9, len(one_set)-15, 5):
        X.append(one_set.iloc[i-9:i+6, 1:].values)
        y.append(one_set.iloc[i]['updown'])

X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(71832, 14, 7) (71832,)


In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

Training set size: 57465
Validation set size: 7183
Testing set size: 7184
(57465, 14, 7)
(7183, 14, 7)
(7184, 14, 7)


In [29]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)